In [3]:
from sqlalchemy import create_engine

engine = create_engine("sqlite+pysqlite:///database.sqlite", echo=True)

In [5]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [6]:
from sqlalchemy import Table, Column, Integer, String

user_table = Table(
    "user_account",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("name", String(30)),
    Column("fullname", String),
    extend_existing=True
)

In [7]:
from sqlalchemy import ForeignKey

address_table = Table(
    "address",
    metadata_obj,
    Column("id", Integer, primary_key=True),
    Column("user_id", ForeignKey("user_account.id"), nullable=False),
    Column("email_address", String, nullable=False),
    extend_existing=True
)

In [8]:
from sqlalchemy import update

stmt = (
    update(user_table)
    .where(user_table.c.name == "patrick")
    .values(fullname="Patrick the Star")
)
print(stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1


In [9]:
stmt = update(user_table).values(fullname="Username: " + user_table.c.name)
print(stmt)

UPDATE user_account SET fullname=(:name_1 || user_account.name)


In [12]:
from sqlalchemy import bindparam

stmt = (
    update(user_table)
    .where(user_table.c.name == bindparam("oldname"))
    .values(name=bindparam("newname"))
)
print(stmt)

UPDATE user_account SET name=:newname WHERE user_account.name = :oldname


In [14]:
with engine.begin() as conn:
    conn.execute(
        stmt,
        [
            {"oldname": "jack", "newname": "ed"},
            {"oldname": "wendy", "newname": "mary"},
            {"oldname": "jim", "newname": "jake"},
        ],
    )

2024-12-25 15:02:35,367 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-25 15:02:35,368 INFO sqlalchemy.engine.Engine UPDATE user_account SET name=? WHERE user_account.name = ?
2024-12-25 15:02:35,369 INFO sqlalchemy.engine.Engine [cached since 11.64s ago] [('ed', 'jack'), ('mary', 'wendy'), ('jake', 'jim')]
2024-12-25 15:02:35,370 INFO sqlalchemy.engine.Engine COMMIT


In [16]:
from sqlalchemy import select

scalar_subq = (
    select(address_table.c.email_address)
    .where(address_table.c.user_id == user_table.c.id)
    .order_by(address_table.c.id)
    .limit(1)
    .scalar_subquery()
)
update_stmt = update(user_table).values(fullname=scalar_subq)
print(update_stmt)

UPDATE user_account SET fullname=(SELECT address.email_address 
FROM address 
WHERE address.user_id = user_account.id ORDER BY address.id
 LIMIT :param_1)


In [17]:
update_stmt = (
    update(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == "patrick@aol.com")
    .values(fullname="Pat")
)
print(update_stmt)

UPDATE user_account SET fullname=:fullname FROM address WHERE user_account.id = address.user_id AND address.email_address = :email_address_1


In [18]:
update_stmt = (
    update(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == "patrick@aol.com")
    .values(
        {
            user_table.c.fullname: "Pat",
            address_table.c.email_address: "pat@aol.com",
        }
    )
)

In [19]:
from sqlalchemy.dialects import mysql
print(update_stmt.compile(dialect=mysql.dialect()))

UPDATE user_account, address SET address.email_address=%s, user_account.fullname=%s WHERE user_account.id = address.user_id AND address.email_address = %s


In [22]:
some_table = Table(
    "some_table",
    metadata_obj,
    Column("x", Integer),
    Column("y", Integer),
    extend_existing=True
)

In [23]:
update_stmt = update(some_table).ordered_values((some_table.c.y, 20), (some_table.c.x, some_table.c.y + 10))
print(update_stmt)

UPDATE some_table SET y=:y, x=(some_table.y + :y_1)


In [24]:
from sqlalchemy import delete
stmt = delete(user_table).where(user_table.c.name == "patrick")
print(stmt)

DELETE FROM user_account WHERE user_account.name = :name_1


In [25]:
delete_stmt = (
    delete(user_table)
    .where(user_table.c.id == address_table.c.user_id)
    .where(address_table.c.email_address == "patrick@aol.com")
)
from sqlalchemy.dialects import mysql
print(delete_stmt.compile(dialect=mysql.dialect()))

DELETE FROM user_account USING user_account, address WHERE user_account.id = address.user_id AND address.email_address = %s


In [26]:
with engine.begin() as conn:
    result = conn.execute(
        update(user_table)
        .values(fullname="Patrick McStar")
        .where(user_table.c.name == "patrick")
    )
    print(result.rowcount)

2024-12-25 15:28:45,663 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-25 15:28:45,666 INFO sqlalchemy.engine.Engine UPDATE user_account SET fullname=? WHERE user_account.name = ?
2024-12-25 15:28:45,667 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ('Patrick McStar', 'patrick')
2
2024-12-25 15:28:45,671 INFO sqlalchemy.engine.Engine COMMIT


In [27]:
update_stmt = (
    update(user_table)
    .where(user_table.c.name == "patrick")
    .values(fullname="Patrick the Star")
    .returning(user_table.c.id, user_table.c.name)
)
print(update_stmt)
delete_stmt = (
    delete(user_table)
    .where(user_table.c.name == "patrick")
    .returning(user_table.c.id, user_table.c.name)
)
print(delete_stmt)

UPDATE user_account SET fullname=:fullname WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name
DELETE FROM user_account WHERE user_account.name = :name_1 RETURNING user_account.id, user_account.name
